In [21]:
import sys
sys.path.insert(0, '../BO')

In [22]:
from gp import *
import numpy as np
from sklearn.datasets import fetch_mldata
from BN import *

In [23]:
# Load and format the training and testing data
mnist = fetch_mldata('MNIST original')

In [24]:
BN = BN(mnist)

In [25]:
loss = BN.train_bayesian_nn

In [26]:
bounds = np.array([[-15, 0.0], [10., 150.], [5., 2000.], [1., 5.], [0.01, 5.], [0.01, 5.], [0.01, 5.], [0.01, 5.], [0.01, 5.]])
# maxeps specifies how many BOs we want to run
maxeps = 6
# n_iters is number of iterations we want for each run of BO
n_iters = 100
acqui_eva_num = 5
greater_is_better = False
n_pre_samples = 5
coor_sigma = 1 * np.array([0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2])
burn_in = 20
input_dimension = 9
mode = 'MAP'
acqui_mode = 'MCMC'
acqui_sample_num = 5
process_sample_mode = 'abs'
prior_mode ='normal_prior'
likelihood_mode = 'normal_likelihood'

In [ ]:
y_list = list()
time_list = list() 
for i in range(n_iters):
    y_list.append(list())


for j in range(maxeps):
    print ('Running %d episode' % (j + 1))
    xp, yp, timep = bayesian_optimisation(coor_sigma = coor_sigma, 
                                   burn_in = burn_in, 
                                   input_dimension = input_dimension,
                                   n_iters = n_iters, 
                                   sample_loss = loss, 
                                   bounds = bounds, 
                                   n_pre_samples = n_pre_samples, 
                                   acqui_eva_num = acqui_eva_num,  
                                   greater_is_better=greater_is_better, 
                                   mode = mode, 
                                   acqui_mode = acqui_mode, 
                                   acqui_sample_num = acqui_sample_num,   
                                   process_sample_mode = process_sample_mode, 
                                   prior_mode = prior_mode, 
                                   likelihood_mode = likelihood_mode)
    y_train = yp[n_pre_samples:]
    for idx in range(len(y_train)):
        if idx == 0:
            y_list[idx].append(y_train[idx])
        else:
            y_list[idx].append(np.min(y_train[:idx]))
    for i in range(len(y_train)):
        time_list[i].append(timep[i])

Running 1 episode
Start presampling...
	Learning rate: 6.41967057582e-07, training epochs: 82, batch size: 1567, n_samples: 3, prior_sigma: 3.4188635, init_sigma_weights_mu:  2.3055563, init_sigma_biases_mu: 2.849934, init_sigma_weights_rho: 0.49737722, init_sigma_biases_rho: 4.8458924
Accuracy 0.0931428571429
	Learning rate: 2.07800050979e-05, training epochs: 78, batch size: 1559, n_samples: 4, prior_sigma: 2.264739, init_sigma_weights_mu:  2.8584397, init_sigma_biases_mu: 4.8499064, init_sigma_weights_rho: 1.8719058, init_sigma_biases_rho: 0.72643185
Accuracy 0.126666666667
	Learning rate: 1.04713872982e-06, training epochs: 78, batch size: 1406, n_samples: 1, prior_sigma: 1.6754179, init_sigma_weights_mu:  4.1364236, init_sigma_biases_mu: 0.74344105, init_sigma_weights_rho: 0.27249765, init_sigma_biases_rho: 3.5490997
Accuracy 0.118285714286
	Learning rate: 1.63699295247e-06, training epochs: 82, batch size: 1129, n_samples: 1, prior_sigma: 1.7427502, init_sigma_weights_mu:  1.9390

In [4]:
def return_mean_std(y_list):
    mean_list = list()
    std_list = list()
    for one_list in y_list:
        mean_list.append(np.mean(one_list))
        std_list.append(np.std(one_list))
    return mean_list, std_list
mean_list_MCMC, std_list_MCMC = return_mean_std(y_list)

In [ ]:
import matplotlib.pyplot as plt
x_MCMC = [i+1 for i in range(len(y_list))]

plt.errorbar(x_MCMC, mean_list_MCMC, yerr = std_list_MCMC, fmt = '-')

plt.show()

In [7]:
import pickle
opt_dict = {}
opt_dict['y_list'] = y_list
with open('BN_MCMC_ABS.pkl','wb') as file:
    pickle.dump(opt_dict, file)

In [8]:
with open('BN_MCMC_ABS.pkl', 'rb') as pickle_file:
    data1 = pickle.load(pickle_file)